In [1]:
!pip install nltk

In [2]:
import pandas as pd

In [11]:
# en_gt = pd.read_csv('../ACTER/en/htfl/annotations/htfl_en_terms_nes.ann', sep='	', engine='python',header=None)
# en_preds = pd.read_csv('../results/en/bert_en_nes_predictions.csv')
en_gt = pd.read_csv('../ACTER/fr/htfl/annotations/htfl_fr_terms_nes.ann', sep='	', engine='python',header=None)
en_preds = pd.read_csv('../results/fr/camembert_fr_nes_predictions.csv')
preds = list(en_preds['predictions'])
gts = list(en_gt[0])

In [12]:
from collections import Counter
preds_len = [len(x.split(' ')) for x in preds]
gts_len = [len(x.split(' ')) for x in gts]
print(Counter(preds_len))
print(Counter(gts_len))

Counter({1: 930, 2: 431, 3: 114, 4: 22, 5: 3, 6: 2})
Counter({1: 1170, 2: 800, 3: 376, 4: 144, 5: 40, 6: 31, 7: 8, 8: 4, 10: 4, 13: 2, 11: 2, 29: 1, 14: 1, 9: 1, 21: 1})


In [13]:
max(gts, key=len)

'a comparison of outcomes in patients in new york heart association [nyha] class ii heart failure when treated with eplerenone or placebo in addition to standard heart failure medicines'

In [4]:
# P = TP/(TP+FP)
# R =  TP/(TP+FN)
def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP, FP, FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score

precision, recall, f1_score = evaluation_metrics(preds, gts)
print(len(set(preds)))
print(len(set(gts)))
precision, recall, f1_score

1090 319 1271
1409
2361


(77.36, 46.17, 57.83)

In [5]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
port = PorterStemmer()
wnl = WordNetLemmatizer()

preds_stem = [port.stem(i) for i in preds]
gts_stem = [port.stem(i) for i in gts]

print('After stemming')
print(len(set(preds_stem)))
print(len(set(gts_stem)))
print(evaluation_metrics(list(set(preds_stem)), list(set(gts_stem))))

preds_lem = [wnl.lemmatize(i) for i in preds]
gts_lem = [wnl.lemmatize(i) for i in gts]

print('After lemmatization')
print(len(set(preds_lem)))
print(len(set((gts_lem))))
print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

preds_com = [port.stem(wnl.lemmatize(i)) for i in preds]
gts_com = [port.stem(wnl.lemmatize(i)) for i in gts]

print('After combined')
print(len(set(preds_com)))
print(len(set((gts_com))))
evaluation_metrics(list(set(preds_com)), list(set(gts_com)))

preds_ =[x for x in preds if len(x) >= 2]
gts_ =[x for x in gts if len(x) >= 2]
print(len(set(preds_)))
print(len(gts))
evaluation_metrics(set(preds_), gts)

After stemming
1259
2103
953 306 1150
(75.69, 45.32, 56.69)
After lemmatization
1351
2279
1033 318 1246
(76.46, 45.33, 56.92)
After combined
1258
2102
952 306 1150
1394
2361
1088 306 1273


(78.05, 46.08, 57.95)

In [ ]:
fr_gt = pd.read_csv('../ACTER/nl/htfl/annotations/htfl_nl_terms.ann', sep='	', engine='python',header=None)
fr_preds = pd.read_csv('../results/nl/bert_nl_ann_predictions.csv')
preds = list(fr_preds['predictions'])
gts = list(fr_gt[0])

In [ ]:
precision, recall, f1_score = evaluation_metrics(preds, gts)

print(len(set(preds)))
print(len(set(gts)))
print(precision, recall, f1_score)

In [ ]:
preds_ =[x for x in preds if len(x) >= 2]
gts_ =[x for x in gts if len(x) >= 2]
print(len(set(preds_)))
print(len(gts))
evaluation_metrics(set(preds_), gts)

In [ ]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
print(stemmer.stem('voudrais'))
print(stemmer.stem('couvre'))
print(stemmer.stem('dors'))
print(stemmer.stem('animaux'))
print(stemmer.stem('yeux'))